## globus connect

* https://www.globusid.org/
* https://www.globus.org/app/activity
* https://docs.globus.org/ 
*  https://docs.globus.org/cli/examples/

## globus cli and globus sdk

* sdk documentation: https://globus-sdk-python.readthedocs.io/en/stable/
* cli documentation: https://docs.globus.org/cli/
   * add globus cli to PATH
   
### Prerequisite: registered globus app with associated rights

(see globus sdk documentation with respect to registration)

In [1]:
# replication_test app registered in ESGF_DKRZ globus project space
# get the client id: https://developers.globus.org/   ( https://globus-sdk-python.readthedocs.io/en/stable/tutorial/ )
# clientid = 933535bb-a28d-4ced-af57-da6bebd4e0a6

import globus_sdk
CLIENT_ID = '933535bb-a28d-4ced-af57-da6bebd4e0a6'
client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow()

authorize_url = client.oauth2_get_authorize_url()
print('Please go to this URL and login: {0}'.format(authorize_url))


Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=933535bb-a28d-4ced-af57-da6bebd4e0a6&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=BFhgCOAVunVZZciHZN9Q_FuK2gQ8DAR3CL1fUFhClbM&code_challenge_method=S256&access_type=online


In [2]:
auth_code ='RpKXEnhEG91Pv0EB6OsurNkoeebTnx'
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

globus_auth_data = token_response.by_resource_server['auth.globus.org']
globus_transfer_data = token_response.by_resource_server['transfer.api.globus.org']

# most specifically, you want these tokens as strings
AUTH_TOKEN = globus_auth_data['access_token']
TRANSFER_TOKEN = globus_transfer_data['access_token']

In [3]:
# a GlobusAuthorizer is an auxiliary object we use to wrap the token. In
# more advanced scenarios, other types of GlobusAuthorizers give us
# expressive power
authorizer = globus_sdk.AccessTokenAuthorizer(TRANSFER_TOKEN)
tc = globus_sdk.TransferClient(authorizer=authorizer)

# high level interface; provides iterators for list responses
print("My Endpoints:")
for ep in tc.endpoint_search(filter_scope="my-endpoints"):
    print("[{}] {}".format(ep["id"], ep["display_name"]))

My Endpoints:
[7c32e150-6f02-11e8-9327-0a6d4e044368] cmip-comp2-dkrz
[ff780758-6eed-11e8-9327-0a6d4e044368] stephan_dkrz_lab


In [7]:
endpoint_name = "LLNL AIMS ESGF DTNs"
eps = tc.endpoint_search(endpoint_name, fields="id,display_name")
endpoint_id = None
for ep in eps:
    if ep["display_name"] == endpoint_name:
        endpoint_id = ep["id"]
        break
print("LLNL endpoint:", endpoint_id)        

LLNL endpoint: dd1ee76f-6d04-11e5-ba46-22000b92c6ec


In [6]:
endpoint_name = "cmip-transfer2"
eps = tc.endpoint_search(endpoint_name, fields="id,display_name")
endpoint_id = None
for ep in eps:
    if ep["display_name"] == endpoint_name:
        endpoint_id = ep["id"]
        break
print("LLNL endpoint:", endpoint_id) 

LLNL endpoint: None


In [9]:
source_id = "dd1ee76f-6d04-11e5-ba46-22000b92c6ec"
dest_id = "86ce3984-7ae2-11e8-9443-0a6d4e044368"
tc.endpoint_autoactivate(source_id)
tc.endpoint_autoactivate(dest_id)

# This does not exactly match -a, for example it cannot preserve permissions or ownership.
tdata = globus_sdk.TransferData(tc, source_id, dest_id,
                                preserve_timestamp=True)

source_dir = "/~/css03_data/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/amip/r5i1p1f1/Amon/hfls/gn/v20181016/"
target_dir = "/data/transfer/import"
tdata.add_item(source_dir, target_dir, recursive=True)

submit_result = tc.submit_transfer(tdata)

# wait until transfer is complete, by polling every 15 seconds
import time

poll_interval = 15 # how long to sleep (in seconds) between polls
max_wait = 360 # only wait for up to this many seconds
wait_time = 0 # how long we've waited for the task so far

while True:
    status = tc.get_task(submit_result["task_id"], fields="status")["status"]
    if status in ("SUCCEEDED", "FAILED"):
        print("Task completed with status", status)
        break
        
    if wait_time >= max_wait:
        print("Task did not complete before max wait time")
        break
    
    print("Task not yet complete (status {}), sleeping for {} seconds...".format(
          status, poll_interval))
    time.sleep(poll_interval)
    wait_time += poll_interval

Task not yet complete (status ACTIVE), sleeping for 15 seconds...
Task completed with status SUCCEEDED


### test of globbus connect personal and old ESGF portal globus command line access

* Installation: https://www.globus.org/globus-connect-personal 
* myproxy: systemctl status myproxy-server.service

In [ ]:
### globus connect personal use on laptop with esgf portal:
### pip install globus-cli or globus-sdk 

./globusconnectpersonal -start

globus login
globus endpoint search --filter-scope my-endpoints
globus endpoint search --filter-scope my-endpoints
globus transfer --help

download script
python globus_download_20180613_062251.py -e stephank16#7c32e150-6f02-11e8-9327-0a6d4e044368 -u stephank16 -s .ssh/github_rsa

NOTICE: This CLI (cli.globusonline.org) has been deprecated and support
will be discontinued on August 1 2018.  Please use the GLobus CLI
(https://docs.globus.org/cli/), a standalone client application.
If you have any questions or need help migrating to the new CLI,
contact support@globus.org.

Task ID: 43a8c0ac-7ba7-11e8-944f-0a6d4e044368

    -- and only works for python2.7 !!

In [ ]:
### globus connect personal on cmip-comp2 

export PATH=/home/dkrz/k202015/anaconda3/bin:$PATH
globus endpoint create --personal cmip-comp2-dkrz
./globusconnectpersonal -start
globus login

 ./globusconnectpersonal -setup 6ef533e6-9b51-4043-9ae3-950462a99f98

globus endpoint create --personal cmip-comp2-dkrz
globus endpoint search --filter-scope my-endpoints



globus  ls 4981cd16-d651-11e6-9ccd-22000a1e3b52:/cmip6_dc1/CMIP6/CMIP/
globus transfer 4981cd16-d651-11e6-9ccd-22000a1e3b52:/cmip6_dc1/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/piControl/r1i1p1f1/Emon/ocontempdiff/gn/v20180314/ 7c32e150-6f02-11e8-9327-0a6d4e044368  
globus transfer  --recursive 4981cd16-d651-11e6-9ccd-22000a1e3b52:/cmip6_dc1/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/piControl/r1i1p1f1/Emon/ocontempdiff/gn/v20180314/ 7c32e150-6f02-11e8-9327-0a6d4e044368
globus transfer  --recursive 4981cd16-d651-11e6-9ccd-22000a1e3b52:/cmip6_dc1/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/piControl/r1i1p1f1/Emon/ocontempdiff/gn/v20180314/ 7c32e150-6f02-11e8-9327-0a6d4e044368:/home/dkrz/k202015


### globus connect server

* Installation: https://docs.globus.org/globus-connect-server-installation-guide/ 

   *  sudo curl -LOs https://downloads.globus.org/toolkit/globus-connect-server/globus-connect-server-repo-latest.noarch.rpm
     
   *  sudo rpm --import https://downloads.globus.org/toolkit/gt6/stable/repo/rpm/RPM-GPG-KEY-Globus
     
   *   sudo yum install globus-connect-server-repo-latest.noarch.rpm
     
   *  sudo yum install globus-connect-server
     
   *  sudo yum install bzip2
     
   *  sudo vi /etc/globus-connect-server.conf
     
   *  sudo globus-connect-server-setup
     
* ESGF AA: https://www.earthsystemcog.org/projects/cog/dtn_setup     
     